# Hello world

In [5]:
import os
import openai

def GPT_Completion(texts):## Call the API key under your account (in a secure way)
    openai.api_key = "***REMOVED***"

    response = openai.Completion.create(engine="text-davinci-002",
                                        prompt =  texts,
                                        temperature = 0.6,
                                        top_p = 1,
                                        max_tokens = 64,
                                        frequency_penalty = 0,
                                        presence_penalty = 0)
    return print(response.choices[0].text)

recipe = 'Provide a cooking recipe based on the following ingredients: \\n \nApple \
          \n \nFlour \
          \n \nChicken \
          \n \nSalt'

GPT_Completion(recipe)



Apple Chicken:

1. Preheat oven to 375 degrees.
2. Coat chicken in flour.
3. Place chicken in a baking dish.
4. Cut apples into thin slices and place on top of chicken.
5. Sprinkle with salt.
6. Bake for 30 minutes


# Generate training dataset

## Generate problems

In [3]:
import os
import filecmp

## cleaning work directories 
!rm dataset/problems/\* &> /dev/null
!rm dataset/plans/\* &> /dev/null
!rm dataset/plans_ipc/\* &> /dev/null
!rm dataset/formatted/\* &> /dev/null

## 8000 data points should improve performance between 2x and 4x
%run generator/pddl_confgen.py 4 9000 -r 24 -co
!mv -v generator/problems/conditional/* dataset/problems > /dev/null

## clean duplicates

problems = sorted(os.listdir('./dataset/problems/'))

duplicates = 0
to_be_removed = []

for i, problem_i in enumerate(problems):
    print("Cleaning --- " + str(round((i / len(problems) * 100), 1)) +"%", end = '\r')
    for problem_j in problems[i + 1:]:
        if filecmp.cmp('./dataset/problems/' + problem_i, 
                       './dataset/problems/' + problem_j, 
                       shallow=False):
            duplicates += 1
            to_be_removed.append(problem_j)
            
for problem in to_be_removed:
    try:
        os.remove('./dataset/problems/' + problem)
    except FileNotFoundError:
        continue
            
print(str(duplicates) + " duplicates deleted")

Writing conditional problem files to /home/alessio/planning/teriyaki/generator/problems/conditional...
Done.
All operations completed. Happy planning!
40 duplicates deleted


## Generate plans with Probe

In [4]:
import os
import subprocess

problems = sorted(os.listdir('./dataset/problems/'))


for i, problem in enumerate(problems):
    
    result = subprocess.run(['./generator/probe', 
                             '-d', './dataset/domains/domain_m_25_B.pddl', 
                             '-i', './dataset/problems/' + problem,
                             '-o', './dataset/plans/m_25_B-' + str(problem.split('_',2)[2:][0])], 
                             stdout=subprocess.PIPE)
    print("Planning --- " + str(round((i / len(problems) * 100), 1)) +"%", end = '\r')
    
#     # temporarly limit to 400 samples
#     if i > 500: break

print("Planning complete")

terminate called after throwing an instance of 'std::bad_alloc'
  what():  std::bad_alloc


terminate called after throwing an instance of 'std::bad_alloc'
  what():  std::bad_alloc


Planning complete0%


## Convert Probe plans to IPC format

In [1]:
import os

plans = sorted(os.listdir('./dataset/plans/'))

for plan in plans:
    probe = open('./dataset/plans/' + plan, "r")
    val = open('./dataset/plans_ipc/' + plan.split(".")[0] + ".plan", 'w+')
    replacement = ""
    # using the for loop
    action_timing = 1
    for line in probe:
        line = line.lower()
        line = '%.3f' % (action_timing / 1000) + "00: " + line
        val.write(line)
        action_timing += 2

    probe.close()
    val.close()
    
print("Plans converted to IPC format")

Plans converted to IPC format


## Validate and remove invalid plans using VAL Validate
This step takes a lot of time and usually all plans are validated. **It can be skipped**.

In [2]:
import os
import subprocess

plans = sorted(os.listdir('./dataset/plans_ipc/'))
invalid_plans = 0

for i, plan in enumerate(plans):
    result = subprocess.run(['./generator/Validate', 
                             './dataset/domains/domain_m_25_B.pddl', 
                             './dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl",
                             './dataset/plans_ipc/' + plan], 
                             stdout=subprocess.PIPE)
    if "Plan failed to execute" in str(result):
        invalid_plans += 1
#         os.remove('./dataset/plans_ipc/m_25_B-' + problem.split('_',2)[2:][0].split(".")[0] + ".plan")
#         os.remove('./dataset/problems/' + problem)

    print("Validating --- " + str(round((i / len(plans) * 100), 1)) +"%", end = '\r')

print('%i plans could not be validated and have been removed from the dataset' % invalid_plans)

6 plans could not be validated and have been removed from the dataset


## Sort problem & plans in batch folders

In [19]:
import os
import shutil

## cleaning work directories 
!rm dataset/batches/500/problems/* &> /dev/null
!rm dataset/batches/500/plans/* &> /dev/null
!rm dataset/batches/1000/problems/* &> /dev/null
!rm dataset/batches/1000/plans/* &> /dev/null
!rm dataset/batches/2000/problems/* &> /dev/null
!rm dataset/batches/2000/plans/* &> /dev/null
!rm dataset/batches/4000/problems/* &> /dev/null
!rm dataset/batches/4000/plans/* &> /dev/null
!rm dataset/batches/8000/problems/* &> /dev/null
!rm dataset/batches/8000/plans/* &> /dev/null
!rm dataset/batches/validation/problems/* &> /dev/null
!rm dataset/batches/validation/plans/* &> /dev/null

## fetching and sorting problems and plans

problems = sorted(os.listdir('./dataset/problems/'))
plans = sorted(os.listdir('./dataset/plans_ipc/'))

for i, plan in enumerate(plans):
    if i < 500:
        shutil.copy2('./dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", 
                     'dataset/batches/500/problems/')
        shutil.copy2('./dataset/plans_ipc/' + plans[i], 'dataset/batches/500/plans/')
    elif i < 1000:
        shutil.copy2('./dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", 
                     'dataset/batches/1000/problems/')
        shutil.copy2('./dataset/plans_ipc/' + plans[i], 'dataset/batches/1000/plans/')
    elif i < 2000:
        shutil.copy2('./dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", 
                     'dataset/batches/2000/problems/')
        shutil.copy2('./dataset/plans_ipc/' + plans[i], 'dataset/batches/2000/plans/')
    elif i < 4000:
        shutil.copy2('./dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", 
                     'dataset/batches/4000/problems/')
        shutil.copy2('./dataset/plans_ipc/' + plans[i], 'dataset/batches/4000/plans/')
    elif i < 8000:
        shutil.copy2('./dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", 
                     'dataset/batches/8000/problems/')
        shutil.copy2('./dataset/plans_ipc/' + plans[i], 'dataset/batches/8000/plans/')
    else:
        shutil.copy2('./dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", 
                     'dataset/batches/validation/problems/')
        shutil.copy2('./dataset/plans_ipc/' + plans[i], 'dataset/batches/validation/plans/')

print('All problems and plans have been sorted into batches.')

All problems and plans have been sorted into batches.


## Compile GPT3 training datasets

This step will compile `jsonl` for all training batches. Later you can use each of them to increment the fine tuning dataset size progressively.

Unfortunately, before use you need to manually run in a terminal the following line for each of them:

```>openai tools fine_tunes.prepare_data -f './dataset/batches/[YOUR_BATCH]/teriyaki.jsonl```

=====================================================

TODO: `joint3)\n)\n)\n` is seen as part of the prompt

From the documentation regarding conditional generation:
* Use a separator at the end of the prompt, e.g. \n\n###\n\n. Remember to also append this separator when you eventually make requests to your model.
* Use an ending token at the end of the completion, e.g. END
* Remember to add the ending token as a stop sequence during inference, e.g. stop=[" END"]

Example:
```{"prompt":"<Product Name>\n<Wikipedia description>\n\n###\n\n", "completion":" <engaging ad> END"}```

In [21]:
import os
import subprocess

batches = ["500", "1000", "2000", "4000", "8000", "validation"]

for batch in batches:
    dataset = open('./dataset/batches/' + batch + '/teriyaki.jsonl', 'w+')
    plans = sorted(os.listdir('./dataset/batches/' + batch + '/plans/'))

    # extracting prompts from problems and completions from IPC plans 
    for i, plan in enumerate(plans):
        plan_f = open('./dataset/batches/' + batch + '/plans/' + plan, "r")
        problem_f = open('./dataset/batches/' + batch + '/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", "r")
        lines = problem_f.readlines()

        # lines[49:62] is the range of meaningful predicates at initial state
        dataset.write(repr('{"prompt":"\n(:init\n' + ''.join(lines[49:-1]).replace("    ", "") + 
              '\n\n###\n\n", "completion":" ' + plan_f.read().replace("    ", "") + ' END"}')[1:-1] + "\n")

    dataset.close()

print("Datasets ready!")
print("Before training, run in a terminal\n\t>openai tools fine_tunes.prepare_data -f './dataset/batches/[YOUR_BATCH]/teriyaki.jsonl'")

Datasets ready!
Before training, run in a terminal
	>openai tools fine_tunes.prepare_data -f './dataset/batch/[YOUR_BATCH]/teriyaki.jsonl'


## Compute training cost
`1 token ~= 4 chars
0.0300$/1K Tokens`

In practice it is ~15$ per 1000 samples for this specific planning domain

In [23]:
import os

dataset = open('./dataset/batches/500/teriyaki.jsonl', 'r')
chars = len(dataset.read())
dataset.close()
tokens = chars / 4
cost = tokens * 0.00003

print("Training this dataset will consume about {0} tokens for an estimated cost of {1}$".format(tokens, round(cost,2)))

Training this dataset will consume about 157412.5 tokens for an estimated cost of 4.72$


## Creating a fine-tuned model!

### <ins>RUN ONCE</ins>

### Last model
Job **ft-PXtNnUR8jjnAzIgW1YhRpGrt**

Model **davinci:ft-teriyaki:teriyaki-001-2022-11-30-18-48-47**

### Dataset 
From the documentation regarding conditional generation:
* Aim for at least ~500 examples
* Ensure that the prompt + completion doesn't exceed 2048 tokens, including the separator
* Ensure the examples are of high quality and follow the same desired format
* Ensure that the dataset used for finetuning is very similar in structure and type of task as what the model will be used for
* Using Lower learning rate and only 1-2 epochs tends to work better for these use cases

LEARN MORE ABOUT VALIDATION, DELETING, AND SUFFIX @ 

https://beta.openai.com/docs/guides/fine-tuning/advanced-usage

https://beta.openai.com/docs/api-reference/fine-tunes

### Training hyperparameters
* `model` use `'davinci'` or `'code-davinci-002'` (CODEX, not yet supported)
* `n_epochs` defaults to 4. The number of epochs to train the model for. An epoch refers to one full cycle through the training dataset. For generative use cases 2 epochs is generally better.
* `batch_size` defaults to ~0.2% of the number of examples in the training set, capped at 256. Larger batch sizes tend to work better for larger datasets.
* `learning_rate_multiplier`defaults to 0.05, 0.1, or 0.2 depending on final `batch_size`. We recommend experimenting with values in the range 0.02 to 0.2. Larger learning rates often perform better with larger batch sizes.
* `compute_classification_metrics`defaults to False. If True, for fine-tuning for classification tasks, computes classification-specific metrics (accuracy, F-1 score, etc) on the validation set at the end of every epoch.

### Results
The _results.csv file contains a row for each training step, where a step refers to one forward and backward pass on a batch of data. In addition to the step number, each row contains the following fields corresponding to that step:

* `elapsed_tokens`: the number of tokens the model has seen so far (including repeats)
* `elapsed_examples`: the number of examples the model has seen so far (including repeats), where one example is one element in your batch. For example, if batch_size = 4, each step will increase elapsed_examples by 4.
* `training_loss`: loss on the training batch
* `training_sequence_accuracy`: the percentage of completions in the training batch for which the model's predicted tokens matched the true completion tokens exactly. For example, with a batch_size of 3, if your data contains the completions [[1, 2], [0, 5], [4, 2]] and the model predicted [[1, 1], [0, 5], [4, 2]], this accuracy will be 2/3 = 0.67
* `training_token_accuracy`: the percentage of tokens in the training batch that were correctly predicted by the model. For example, with a batch_size of 3, if your data contains the completions [[1, 2], [0, 5], [4, 2]] and the model predicted [[1, 1], [0, 5], [4, 2]], this accuracy will be 5/6 = 0.83

In [80]:
import os
import openai
import requests

# !openai -k ***REMOVED*** \
# api fine_tunes.create -t './dataset/batches/500/teriyaki.jsonl' \
# -v './dataset/batches/validation/teriyaki.jsonl' \
# -m 'davinci' --suffix "teriyaki-001" --n_epochs 2

!openai -k ***REMOVED*** \
api fine_tunes.follow -i ft-JNw1L5bGAnUwJdxSXQAQrdbj

[2022-11-30 23:20:17] Created fine-tune: ft-JNw1L5bGAnUwJdxSXQAQrdbj
[2022-11-30 23:20:25] Fine-tune costs $109.78
[2022-11-30 23:20:25] Fine-tune enqueued. Queue number: 5
[2022-11-30 23:25:17] Fine-tune is in the queue. Queue number: 4
[2022-12-01 00:08:39] Fine-tune is in the queue. Queue number: 3
[2022-12-01 00:12:43] Fine-tune is in the queue. Queue number: 2
[2022-12-01 00:13:08] Fine-tune is in the queue. Queue number: 1
[2022-12-01 00:22:56] Fine-tune is in the queue. Queue number: 0
[2022-12-01 00:24:34] Fine-tune started
[2022-12-01 00:56:47] Completed epoch 1/2

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-JNw1L5bGAnUwJdxSXQAQrdbj



## Save results

In [66]:
response = !openai -k ***REMOVED*** \
api fine_tunes.results -i ft-PXtNnUR8jjnAzIgW1YhRpGrt > dataset/batches/500/results.csv

response = !openai -k ***REMOVED*** \
api fine_tunes.results -i ft-EFuAgXbaOpytBR8VGcIq4Qfe > dataset/batches/1000/results.csv

response = !openai -k ***REMOVED*** \
api fine_tunes.results -i ft-OkqNqBWMm0ATjHB7r2Prsgfj > dataset/batches/2000/results.csv

response = !openai -k ***REMOVED*** \
api fine_tunes.results -i ft-cldwwOoDt4xGY2WGtTxJo5yh > dataset/batches/4000/results.csv

# response = !openai -k ***REMOVED*** \
# api fine_tunes.results -i ft-PXtNnUR8jjnAzIgW1YhRpGrt > dataset/batches/8000/results.csv

print("Done!")

Done!


## Further tuning

### <ins>Insert previous model and run in order for each batch at most once!</ins>

* **1000** ft-EFuAgXbaOpytBR8VGcIq4Qfe - ft-teriyaki:teriyaki-001-2022-11-30-19-48-12
* **2000** ft-OkqNqBWMm0ATjHB7r2Prsgfj - ft-teriyaki:teriyaki-001-2022-11-30-21-21-23
* **4000** ft-cldwwOoDt4xGY2WGtTxJo5yh - ft-teriyaki:teriyaki-001-2022-11-30-22-07-22
* **8000** ft-JNw1L5bGAnUwJdxSXQAQrdbj - 

Not our case, but...

```if your new training data is much smaller than your previous training data, you may find it useful to reduce learning_rate_multiplier by a factor of 2 to 4.```

In [59]:
import os
import openai
import requests

batches = ["1000", "2000", "4000", "8000"]

!openai -k ***REMOVED*** \
api fine_tunes.create -t './dataset/batches/1000/teriyaki.jsonl' \
-v './dataset/batches/validation/teriyaki.jsonl' \
-m 'davinci:ft-teriyaki:teriyaki-001-2022-11-30-18-48-47' --suffix "teriyaki-001" --n_epochs 2

# api fine_tunes.follow -i ft-PXtNnUR8jjnAzIgW1YhRpGrt

Upload progress: 100%|███████████████████████| 646k/646k [00:00<00:00, 307Mit/s]
Uploaded file from ./dataset/batches/1000/teriyaki.jsonl: file-WkGWR3hD1sofGiz4lDaFCOJ9
Found potentially duplicated files with name 'teriyaki.jsonl', purpose 'fine-tune' and size 1101044 bytes
file-xQOM754uLyXXJezwzs1BLlSZ
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: ^C



## Test it!

In [75]:
import os
import openai

def GPT_Completion(texts):## Call the API key under your account (in a secure way)
    openai.api_key = "***REMOVED***"

    response = openai.Completion.create(engine="davinci:ft-teriyaki:teriyaki-001-2022-11-30-22-07-22",
                                        prompt =  texts,
                                        top_p = 1,       # modify ONLY top_p or temperature, not both       
                                        temperature = 0, # argmax
                                        max_tokens = 600,
                                        presence_penalty = 0, # -2 < x < 2, negative favors repetitions
                                        frequency_penalty = 0,
                                        # best_of = 4, # 3-5 work well for code completion
                                        stop = "END")
    return print(response.choices[0].text)

problem = '(:init\n(angle_joint angle0 joint1)\n(angle_joint angle345 joint2)\n(angle_joint angle0 joint3)\n(in-centre joint3)\n(in-hand link1)\n(in-hand link2)\n(grasp gleft link1)\n(grasp gright link2))\n(:goal(and\n(angle_joint angle285 joint1)\n(angle_joint angle285 joint2)\n(angle_joint angle315 joint3)\n)\n)\n)\n\n\n###\n\n'

GPT_Completion(problem)

 0.00100: (release-links link2 link1 joint1 gleft gright)
0.00300: (link-to-central-grasp link2 link1 joint3 joint1 gleft gright)
0.00500: (decrease_angle_first_child_45 link2 link1 joint1 angle0 angle345 angle330 angle315 gleft gright)
0.00700: (decrease_angle_first_child link2 link1 joint1 angle315 angle300 gleft gright)
0.00900: (decrease_angle_first_child link2 link1 joint1 angle300 angle285 gleft gright)
0.01100: (release-links link2 link1 joint1 gleft gright)
0.01300: (link-to-central-grasp link3 link2 joint1 joint2 gleft gright)
0.01500: (increase_angle_first_child link3 link2 joint2 angle270 angle285 gleft gright)
0.01700: (release-links link3 link2 joint2 gleft gright)
0.01900: (link-to-central-grasp link4 link3 joint2 joint3 gleft gright)
0.02100: (increase_angle_first_child link4 link3 joint3 angle300 angle315 gleft gright)
 


## Run extended tests on the dataset
Plan with Teriyaki for all available problems and run VAL Validate

It reports:
* success rate
* min, max, avg and standard deviation of planning times
* average number of actions before Teryiaki makes a mistake

In [ ]:
import os
import openai

def GPT_Completion(texts):## Call the API key under your account (in a secure way)
    openai.api_key = "***REMOVED***"

    response = openai.Completion.create(engine="davinci:ft-personal-2022-08-18-17-42-22",
                                        prompt =  texts,
                                        top_p = 1,       # modify ONLY top_p or temperature, not both       
                                        temperature = 0, # argmax
                                        max_tokens = 400,
                                        presence_penalty = 0, # -2 < x < 2, negative favors repetitions
                                        frequency_penalty = 0,
                                        # best_of = 4, # 3-5 work well for code completion
                                        stop = "END")
    return response.choices[0].text

batches = ["500", "1000", "2000", "4000", "8000"]

## is planning needed or already in GPT-3 validation?

## Iterative planning trick
* Try to plan only N actions 
* Use VAL **ValStep** to simulate the outcomes of said actions and save it to a problem file
* Replan using the new problem file
* Iterate until goal state is reached